In [1]:
import numpy as np
from tqdm import tqdm
from time import time
import json, pickle, os, string, tqdm, kenlm, json
from collections import defaultdict, Counter
from itertools import groupby
import Levenshtein as Lev
import math

In [2]:
def lse(*args): # logsumexp trick
    a_max = max(args)
    lsp = math.log(sum(math.exp(a-a_max) for a in args))
    return a_max + lsp

In [3]:
#s1 = True text
#s2 = predicted text

def wer_(s1, s2):
    """
    Computes the Word Error Rate, defined as the edit distance between the
    two provided sentences after tokenizing to words.
    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """

    # build mapping of words to integers
    b = set(s1.split() + s2.split())
    word2char = dict(zip(b, range(len(b))))

    # map the words to a char array (Levenshtein packages only accepts
    # strings)
    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    
    return Lev.distance(''.join(w1), ''.join(w2))

def cer_(s1, s2):
    """
    Computes the Character Error Rate, defined as the edit distance.

    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """
    s1, s2, = s1.replace(' ', ''), s2.replace(' ', '')

    return Lev.distance(s1, s2)




#When using the above implementation, use the code belove to calculate the wer in percentatge: 
#pred = list of ouput prediction of model (it is the text) # example [" MY NAME IS HEMANT", " I AM A GOD"]
# total_wer = 0
# for x in range(len(pred)):
#     transcript, reference = data_[x][1], pred[x]
#     wer_inst = wer(transcript, reference)
#     total_wer += float(wer_inst)
# print("WER is : ",total_wer/len(pred),"%")
# wer_(pred,true)/len(pred.split(' '))*100

In [4]:
out = np.load("out.npy")
with open("true.txt", "r") as f:
    reference = f.read()
with open("pred.txt", "r") as f:
    transcript = f.read()

In [5]:
def ctc_best_path(out,labels):
    "implements best path decoding as shown by Graves"
    out = [labels[i] for i in np.argmax(out, axis=1) if i!=labels[-1]]
    o = ""
    for i,j in groupby(out):
        o = o + i
    return o.replace("_","")

In [6]:
labels = "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "

In [7]:
gred_txt = ctc_best_path(out,labels)
print(gred_txt)
wer_(gred_txt,reference)/len(reference.split(' '))*100

OUR PRODATIVITY AND OUR COMMITMENT TO OUR CLIENTS XXXUM TO MAKE SURE THAT WE NEED WHAT BE XXXUM PROMISED FOR THE DEADLINE


19.047619047619047

### WORD LM Implementation

In [8]:
lm_w = kenlm.LanguageModel('/home/hemant/sopi_deep/lm/3_gram_full.arpa')

In [15]:
def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]
        return [i[1] for i in dict_]

#using WORD LM
def ctc_beam_search(out,labels, prune=0.0001, k=20, lm=None,alpha=0.3,beta=12):
    "implements CTC Prefix Search Decoding Algo13.043478260869565%'rithm as shown by Graves"
    '''
    out = ctc output
    labels = string of labels
    prune = prune the ctc output
    k=beam-width
    lm=word age model used
    alpha,beta = hyper-parameters
    '''

    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    out = np.vstack((np.zeros(F), out))
    steps = out.shape[0]
    
    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 1, 0
    prev_beams = ['']
    for t in range(1,steps):
        pruned_alphabet = [labels[i] for i in np.where(out[t] > prune)[0]]
        for b in prev_beams:
            
            for c_t in pruned_alphabet:
                index = labels.index(c_t)
                #Collapsing case (copy case as the last character in the beam)
                if c_t == "_": #Extending with a blank
                    pb[t][b] = out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    continue
                else:
                    i_plus = b + c_t
                    if len(b) > 0 and c_t == b[-1]: #Extending with the same character as the last one
                        pnb[t][b] += out[t][index]*pnb[t-1][b]
                        pnb[t][i_plus] += out[t][index]*pb[t-1][b]
                    #Extending case as the last character is different
                    elif c_t == " " and len(b.replace(' ', '')) > 0 : # LM constraints
                        prob = [i[0] for i in lm.full_scores(i_plus,eos=False,bos=False)][-1]
                        lm_p = (10**prob)**alpha
                        pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])*(len(b.split())+1)**beta
                    else:
                        pnb[t][i_plus] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    # If the new beam is not in the previous beams
                    if i_plus not in prev_beams:
                        pb[t][i_plus] += out[t][labels.index("_")] * (pb[t - 1][i_plus] + pnb[t - 1][i_plus])
                        pnb[t][i_plus] += out[t][index] * pnb[t - 1][i_plus]

        ptot = pb[t] + pnb[t]
        prev_beams = sort_beam(ptot,k)
    return prev_beams[0]

In [16]:
beam_txt = ctc_beam_search(out,labels,0.0001,k=100,lm=lm_w,alpha=1.96,beta=6)
print(beam_txt)
wer_(beam_txt,reference)/len(reference.split(' '))*100

OUR PRODUCTIVITY AND OUR COMMITMENT TO OUR CLIENTS XXXUM TO MAKE SURE THAT WE NEED WHAT WE XXXUM PROMISED FOR THE DEADLINE


9.523809523809524

### CHARACTER LM Implementation


In [17]:
lm_c = kenlm.LanguageModel('/home/hemant/E2E_NER-Through-Speech/LM/3_char_gram.arpa')

In [ ]:
def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]
        return [i[1] for i in dict_]
#using CHARACTER LM
def ctc_beam_search_clm(out,labels, prune=0.001, k=20, lm=None,alpha=0.3,beta=12):
    "implements CTC Prefix Search Decoding Algorithm as shown by Graves"

    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    out = np.vstack((np.zeros(F), out))
    steps = out.shape[0]
    
    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 1, 0
    prev_beams = ['']
    for t in range(1,steps):
        pruned_alphabet = [labels[i] for i in np.where(out[t] > prune)[0]]
        for b in prev_beams:
            for c_t in pruned_alphabet:
                index = labels.index(c_t)
                #Collapsing case (copy case as the last character in the beam)
                if c_t == "_": #Extending with a blank
                    pb[t][b] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])  
                else:  # LM constraints
                    i_plus = b + c_t
                     #Extending with the same character as the last one
                    if len(b) > 0 and c_t == b[-1]:
                        pnb[t][b] += out[t][index]*pnb[t-1][b]
                        pnb[t][i_plus] += out[t][index]*pb[t-1][b]
                    #expanding the beam (extend case as the last character is different)
                    elif len(b.replace(' ', '')) > 0 :
                        prob = [i[0] for i in lm.full_scores(i_plus,eos=False,bos=False)][-1]
                        lm_p = 1#(10**prob)**alpha
                        pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    else:
                        pnb[t][i_plus] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])
        ptot = pb[t] + pnb[t]
        for i in ptot.keys():
            ptot[i] = ptot[i]*(len(i)+1)**beta
        prev_beams = sort_beam(ptot,k)
    return prev_beams

# IMPLEMANTATION

In [18]:
import os
os.chdir("/home/hemant/E2E_NER-Through-Speech/S2T/")
from opts import add_decoder_args, add_inference_args
from utils import load_model
import os
from ctc_decoders import *
import argparse

import numpy as np
import torch
from tqdm import tqdm
from data.data_loader import SpectrogramParser

from opts import add_decoder_args, add_inference_args
from utils import load_model

In [19]:
labels = "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "

In [ ]:
# decoding = 'greedy'
decoding = 'beam_w'
# decoding = 'beam_c'
prune = 0.0001
beam_width = 100
alpha = 1.4
beta = 4
lm = lm_w

torch.set_grad_enabled(False)
device = torch.device("cuda")
model = load_model(device, "/home/hemant/sopi_deep/models/deep/finetuen_sopi.pth")
spect_parser = SpectrogramParser(model.audio_conf, normalize=True)

torch.cuda.set_device(int(0))
with open("/home/hemant/sopi_deep/data/deepspeech/test/data.csv","r") as f:
    csv = f.readlines()

total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
lm = lm_w
output = []
for i in tqdm(csv):
    audio_path, reference_path = i.split(",")

    spect = spect_parser.parse_audio(audio_path).contiguous()
    spect = spect.view(1, 1, spect.size(0), spect.size(1))
    spect = spect.to(device)

    input_sizes = torch.IntTensor([spect.size(3)]).int()
    out, output_sizes = model(spect, input_sizes)
    out = out.cpu().detach().numpy()[0]

    if decoding == "greedy": transcript = ctc_best_path(out,labels)
    elif decoding == "beam_w": transcript = ctc_beam_search(out,labels,prune,beam_width,lm,alpha,beta)
    elif decoding == "beam_c": transcript = ctc_beam_search_clm(out,labels,prune,beam_width,lm,alpha=alpha,beta=beta)
        
    with open(reference_path.replace("\n",""),"r") as f:
        reference = f.readline()
    output.append([transcript,reference])
    wer_inst = wer_(transcript,reference)
    cer_inst = cer_(transcript, reference)
    total_wer += wer_inst
    total_cer += cer_inst
    num_tokens += len(reference.split(' '))
    num_chars += len(reference.replace(' ', ''))
        
wer = (float(total_wer) / num_tokens)*100
cer = (float(total_cer) / num_chars)*100
print('Test Summary \t'
    'Average WER {wer:.3f}\t'
    'Average CER {cer:.3f}\t'.format(wer=wer, cer=cer))


 23%|██▎       | 153/654 [03:53<13:28,  1.61s/it]

greedy 39.850

beam 49.969   1.96 6

In [22]:
def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]
        return [i[1] for i in dict_]

#using WORD LM
def ctc_beam_search(out,labels, prune=0.0001, k=20, lm=None,alpha=0.3,beta=12):
    "implements CTC Prefix Search Decoding Algo13.043478260869565%'rithm as shown by Graves"
    '''
    out = ctc output
    labels = string of labels
    prune = prune the ctc output
    k=beam-width
    lm=word age model used
    alpha,beta = hyper-parameters
    '''

    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    out = np.vstack((np.zeros(F), out))
    steps = out.shape[0]
    
    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 1, 0
    prev_beams = ['']
    for t in range(1,steps):
        pruned_alphabet = [labels[i] for i in np.where(out[t] > prune)[0]]
        for b in prev_beams:
            
            for c_t in pruned_alphabet:
                index = labels.index(c_t)
                #Collapsing case (copy case as the last character in the beam)
                if c_t == "_": #Extending with a blank
                    pb[t][b] = out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    continue
                else:
                    i_plus = b + c_t
                    if len(b) > 0 and c_t == b[-1]: #Extending with the same character as the last one
                        pnb[t][b] += out[t][index]*pnb[t-1][b]
                        pnb[t][i_plus] += out[t][index]*pb[t-1][b]
                    #Extending case as the last character is different
                    elif c_t == " " and len(b.replace(' ', '')) > 0 : # LM constraints
                        prob = [i[0] for i in lm.full_scores(i_plus,eos=False,bos=False)][-1]
                        lm_p = (10**prob)**alpha
                        pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])*(len(b.split())+1)**beta
                    else:
                        pnb[t][i_plus] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    # If the new beam is not in the previous beams
                    if i_plus not in prev_beams:
                        pb[t][i_plus] += out[t][labels.index("_")] * (pb[t - 1][i_plus] + pnb[t - 1][i_plus])
                        pnb[t][i_plus] += out[t][index] * pnb[t - 1][i_plus]

        ptot = pb[t] + pnb[t]
        prev_beams = sort_beam(ptot,k)
    return prev_beams[0]

In [ ]:
torch.set_grad_enabled(False)
device = torch.device("cuda")
model = load_model(device, "/home/hemant/sopi_deep/models/deep/finetuen_sopi.pth")
spect_parser = SpectrogramParser(model.audio_conf, normalize=True)

torch.cuda.set_device(int(0))
with open("/home/hemant/sopi_deep/data/deepspeech/test/data.csv","r") as f:
    csv = f.readlines()
    
# decoding = 'greedy'
decoding = 'beam_w'
# decoding = 'beam_c'
prune = 0.0001
beam_width = 100

In [ ]:
alpha, beta = np.linspace(0.2,2,15), np.linspace(0,6,15)
values = [[i,j] for j in beta for i in alpha]

total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
lm = lm_w
output = []

for alpha,beta in tqdm(values):
    for i in csv:
        audio_path, reference_path = i.split(",")

        spect = spect_parser.parse_audio(audio_path).contiguous()
        spect = spect.view(1, 1, spect.size(0), spect.size(1))
        spect = spect.to(device)

        input_sizes = torch.IntTensor([spect.size(3)]).int()
        out, output_sizes = model(spect, input_sizes)
        out = out.cpu().detach().numpy()[0]

        if decoding == "greedy": transcript = ctc_best_path(out,labels)
        elif decoding == "beam_w": transcript = ctc_beam_search(out,labels,prune,beam_width,lm,alpha,beta)
        elif decoding == "beam_c": transcript = ctc_beam_search_clm(out,labels,prune,beam_width,lm,alpha=alpha,beta=beta)

        with open(reference_path.replace("\n",""),"r") as f:
            reference = f.readline()
        wer_inst = wer_(transcript,reference)
        cer_inst = cer_(transcript, reference)
        total_wer += wer_inst
        total_cer += cer_inst
        num_tokens += len(reference.split(' '))
        num_chars += len(reference.replace(' ', ''))
    np.save("/home/hemant/ctc_decoders/abw.npy",np.array(output))
    wer = (float(total_wer) / num_tokens)*100
    cer = (float(total_cer) / num_chars)*100
    output.append([alpha,beta,wer])
    print("aplha: ",alpha,"beta: ",beta)
    print('Test Summary \t'
        'Average WER {wer:.3f}\t'
        'Average CER {cer:.3f}\t'.format(wer=wer, cer=cer))

In [ ]:
a = []
for i in output:
    a.append(i[2])

In [ ]:
for i in output:
    if i[2] <= min(a):
        print(i)

In [ ]:
np.save("/home/hemant/ctc_decoders/abw.npy",np.array(output))

In [ ]:
# np.load("abw.npy")

In [ ]:
# Test Summary    Average WER 27.562      Average CER 18.691 # beam deafult wer

In [ ]:
# Test Summary 	Average WER 42.638	Average CER 21.520 # beam wrote wer

In [ ]:
# Test Summary 	Average WER 48.410	Average CER 24.264	 greedy matches with the default wer

In [ ]:
# audio_path,reference_path

In [ ]:
# m = audio_path + "," +reference_path
# with open("/home/hemant/junk/out.csv","w") as f:
#     f.write(m)

In [ ]:
# wer_(transcript,reference)/len(reference.split(' '))*100